# Analysing a Simple Circuit using Paricle Swarm Optimzation    ==============================

In this example, we are going to use the `pyswarms` library to analyse a simple circuit by treating it as an optimization problem.The goal of the analysis of the circuit is to find out the current in the circuit.We will use the `pyswarms` library to find an *optimal* solution from a set of candidate solutions.

In [15]:
import sys
# Change directory to access the pyswarms module
sys.path.append('../')

In [16]:
print('Running on Python version: {}'.format(sys.version))

Running on Python version: 3.6.5 (default, Apr  1 2018, 05:46:30) 
[GCC 7.3.0]


In [17]:
# Import modules
import numpy as np

# Import PySwarms
import pyswarms as ps

import math

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Introduction

We will be analysing a Diode-Resistor Circuit in this tutorial.This type of circuit ensures that current would only flow in the direction of the diode's arrow through the resistor(and any load placed in parallel with the resistor).The diode only lets current to flow in one direction through it, namely the direction that it's triangle points. 

PSO algorithm has been used in various fields of electrical engineering for the aim of optimizing such as design of logical circuits, analog and filter design, integrated circuit design, Microwave circuit design etc.


# Determination Of Cost Function                                                                                              
![alt text](https://user-images.githubusercontent.com/39431903/43938822-29aaf9b8-9c66-11e8-8e54-01530db005c6.png "CIRCUIT")

We have used Shockley equation for this tutorial;

<a href="https://www.codecogs.com/eqnedit.php?latex=I&space;=&space;I_s&space;e^{\frac{v_D}{v_T}}" target="_blank"><img src="https://latex.codecogs.com/gif.latex?I&space;=&space;I_s&space;e^{\frac{v_D}{v_T}}" title="I = I_s e^{\frac{v_D}{v_T}}" /></a>

We will simplify the previous equation to get value of diode voltage:

<img src="https://latex.codecogs.com/gif.latex?v_D&space;=&space;v_T&space;\log{\left&space;|\frac{I}{I_s}\right&space;|}" title="v_D = v_T \log{\left |\frac{I}{I_s}\right |}" />

Using the Kirchoff voltage law we get:

![alt text](https://latex.codecogs.com/gif.latex?U&space;=&space;v_D&space;&plus;&space;v_R)

Now we will restructure this equation to get the value of our cost function for the optimization:

![alt text](https://latex.codecogs.com/gif.latex?c&space;=&space;\left&space;|U-v_D-v_R\right&space;|)

The absolute value is necessary because we don't want any negative currents.

![alt text](https://latex.codecogs.com/gif.latex?c&space;=&space;\left&space;|&space;U&space;-&space;v_T&space;\log{\left&space;|&space;\frac{I}{I_s}&space;\right&space;|}&space;-&space;RI&space;\right&space;|)

Hence, diode current is the parameter we want to optimise, 
For this tutorial we will take these values for our parameters:

. U : 10V

. Is : 9.4 pA

. R : 100 Ohm

. Vt : 25.3 mV

Hence, after substituting the values of parameters and  further simplification the cost function will be reduced to: 

![alt text](https://latex.codecogs.com/gif.latex?c&space;=&space;\left&space;|&space;9.24&space;-&space;0.069\left&space;log{\left&space;|&space;I&space;\right&space;|}&space;-&space;100I&space;\right|)
    

# Optimizing our Cost Function

Now as we have or general equation of our cost function we need to define it in python to make it ready for optimisation.


In [20]:
%%time

@pyswarms.cost
def cost_func(I):
    """Method for definition of our cost function
        Inputs:
        --------
        I : current flowing in the diode
        
        Returns:
        ---------
        cost: our cost function
    """
    cost = abs(9.24 - (0.069 * (math.log(I, 10))) - (100 * I))
    return cost

NameError: name 'pyswarms' is not defined